In [5]:
require 'nokogiri'
require 'open-uri'

def parse node, parent_path
  cnts = {}
  node.children
    .find_all{|x| x.class.name == "Nokogiri::XML::Element"}
    .map do |x|
        cnts[x.name] = (cnts[x.name] == nil) ? 1 : cnts[x.name] + 1
        path = parent_path + "/" + x.name + "[#{cnts[x.name]}]"
        [x.name, x.children.size, path, parse(x, path)]
    end
end

def extract_main(url1, url2)
  html =  Nokogiri::HTML(open(url1))
  nodes1 = parse html, "";

  html =  Nokogiri::HTML(open(url2))
  nodes2 = parse html, "";
  
  # body tag 以下を抽出
  nodes = flatten(nodes1,3,[]).find_all{|xs| xs[2] =~ /body/} + flatten(nodes2,3,[]).find_all{|xs| xs[2] =~ /body/}

  # 子要素数が異なるノードを抽出
  diff_nodes = nodes.group_by{|xs| xs[2]}
    .find_all{|xs|xs[1].size != 1}
    .find_all{|xs|xs[1][0][1] != xs[1][1][1]}
    .map{|xs| [xs[0], (xs[1][0][1] - xs[1][1][1]).abs]}

  # メインコンテンツ(差分が最大のノード)を抽出
  diff_nodes.max_by{|xs| xs[1]}
end

def find_images html, root_xpath
  def f node, parent_path
    cnts = {}
    node.children
      .find_all{|x| x.class.name == "Nokogiri::XML::Element"}
      .map do |x|
        cnts[x.name] = (cnts[x.name] == nil) ? 1 : cnts[x.name] + 1
        path = parent_path + "/" + x.name + "[#{cnts[x.name]}]"
        [x.name, path, x, f(x, path)]
      end
  end

  flatten(f(html.at_xpath(root_xpath), root_xpath),3,[]).find_all{|x| x[0] == 'img'}
end

def flatten(nodes, len, r)
  nodes.map{|n|
    r << n[0..(len - 1)]
    flatten(n[-1], len, r)
  }
  r
end


:flatten

In [6]:
require 'json'
#open('input.txt', "w"){|f| f.puts json.to_json}
json = JSON.parse(open('input.txt').read)
p json
json.map do |input|
  # extract main contents
  puts "extracting main contents... "
  main_node = extract_main input["url1"], input["url2"]
  puts "main-contents xpath: " + main_node[0]

  # find images
  puts "finding images"
  html =  Nokogiri::HTML(open(input["url1"]))
  images = find_images(html, main_node[0]).map{|img|
    {xpath:img[1], src:img[2].attribute("src").value}
  }
  puts "finish"

  {title:input["title"], images:images}
end

[{"title"=>"マンガファンタスティック！", "url1"=>"http://manga-fantastic.net/kakegurui-twin/twin-4-2", "url2"=>"http://manga-fantastic.net/keijo/k-9-2"}]
extracting main contents... 
main-contents xpath: /html[1]/body[1]/div[1]/div[1]/div[1]/main[1]/article[1]/div[1]/div[3]
finding images
finish


[{:title=>"マンガファンタスティック！", :images=>[{:xpath=>"/html[1]/body[1]/div[1]/div[1]/div[1]/main[1]/article[1]/div[1]/div[3]/p[1]/img[1]", :src=>"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=="}, {:xpath=>"/html[1]/body[1]/div[1]/div[1]/div[1]/main[1]/article[1]/div[1]/div[3]/noscript[1]/img[1]", :src=>"http://manga-fantastic.net/wp-content/uploads/2017/07/No-0020-4-193x300.png"}, {:xpath=>"/html[1]/body[1]/div[1]/div[1]/div[1]/main[1]/article[1]/div[1]/div[3]/p[13]/img[1]", :src=>"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=="}, {:xpath=>"/html[1]/body[1]/div[1]/div[1]/div[1]/main[1]/article[1]/div[1]/div[3]/noscript[2]/img[1]", :src=>"http://manga-fantastic.net/wp-content/uploads/2017/07/No-0021-4-192x300.png"}, {:xpath=>"/html[1]/body[1]/div[1]/div[1]/div[1]/main[1]/article[1]/div[1]/div[3]/p[19]/img[1]", :src=>"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=="}, {:xpath=>"/html[1]/body[1]/div[1]/div[1]/div[